In [2]:
#  Copyright 2019 Kiril Cvetkov. All Rights Reserved.


import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from keras import backend as K


Using TensorFlow backend.


In [3]:
class LSTMModule(tf.keras.Model):
  def __init__(self, n_filters):
    super(LSTMModule, self).__init__(name='')

    self.lstm1 = LSTM(n_filters)

    
    
  def call(self, inputs, training = False):
    x = self.lstm1(inputs)
    return x


data = tf.convert_to_tensor([0.1,0.2,0.3])

data = K.reshape(data,(1,3,1))

model = LSTMModule(1)
kiko = np.array([0.1,0.2,0.3])
kiko = kiko.reshape(1,3,1)
model.predict(kiko)




array([[-0.07179468]], dtype=float32)

In [4]:
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(1)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))

[[-0.04511731]]


In [5]:
inputs1 = Input(shape=(3, 1))
lstm1_output, state_h, state_c = LSTM(10, return_state=True)(inputs1)
#return state e okay ako vrzuvam edna so druga lstm
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h, state_c])
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
a,b, c = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)
print(c.shape)


[[ True  True  True  True  True  True  True  True  True  True]]
(1, 10)
(1, 10)
(1, 10)


In [10]:
inputs1 = Input(shape=(6, 1))
lstm1_output, state_h, state_c = LSTM(10, return_state=True, return_sequences=True)(inputs1)
lstm1_output = Convolution1D(filters=10, kernel_size=4, strides=1)(lstm1_output)
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h, state_c])
# define input data
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5,0.6]).reshape((1,6,1))
# make and show prediction
a,b, c = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)
print(c.shape)

[[[False False False False False False False False False False]
  [False False False False False False False False False False]
  [False False False False False False False False False False]]]
(1, 3, 10)
(1, 10)
(1, 10)


In [0]:
inputs1 = Input(shape=(3, 1))
lstm1_output, state_h= GRU(10, return_state=True, return_sequences=True)(inputs1)
#return state e okay ako vrzuvam edna so druga lstm
model = Model(inputs=inputs1, outputs=[lstm1_output, state_h])
# define input data
data = np.array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
a,b = model.predict(data)

print(a==b)
print(a.shape)
print(b.shape)



[[[False False False False False False False False False False]
  [False False False False False False False False False False]
  [ True  True  True  True  True  True  True  True  True  True]]]
(1, 3, 10)
(1, 10)


In [54]:
def get_models(words, n_input, n_output, n_units):
  # define training encoder
  #n_input dict_size/hotencoding size
  encoder_inputs = Input(shape=(words, n_input))
  encoder = LSTM(n_units, return_sequences=True)(encoder_inputs)
 
  print(encoder_inputs.shape[1])
  kernel = int(encoder_inputs.shape[1]) - 3 + 1
  print(kernel, kernel)
  #(N, n_input, n_units)
  encoder = Convolution1D(filters=n_units, kernel_size=kernel, strides=1)(encoder)
  
  dense = Dense(n_output, activation='softmax')(encoder)

  model = Model(encoder_inputs, dense)

  # return all models
  return model
  
model = get_models(6, 50,50, 10)


6
4 4


In [0]:
def predict_sequence(inference_encoder, source):
  #encode
  state = inference_encoder.predict(source)
  
  #state.argmax(axis=-1)
  
  print(state[0].shape)
  
  return(state[0])
  
  

In [0]:
from random import *
from keras.utils import to_categorical

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

def get_dataset(n_in, n_out, cardinality, n_samples):
  X1, X2, y = list(), list(), list()
  for _ in range(n_samples):
    # generate source sequence
    source = generate_sequence(n_in, cardinality)
    # define target sequence
    target = source[:n_out]
    target.reverse()
    # create padded input target sequence
    target_in = [0] + target[:-1]
    # encode
#     print(source)
#     print(target)
#     print(target_in)
    src_encoded = to_categorical([source], num_classes=cardinality)
    tar_encoded = to_categorical([target], num_classes=cardinality)
    tar2_encoded = to_categorical([target_in], num_classes=cardinality)
    # store
    X1.append(src_encoded)
    X2.append(tar2_encoded)
    y.append(tar_encoded)
  return np.array(X1), np.array(X2), np.array(y)

In [0]:
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3

In [65]:

# define model
train = get_models(n_steps_in, n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

6
4 4


In [66]:
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
X1 = X1.squeeze()
X2 = X2.squeeze()
y = y.squeeze()
print(X1.shape,X2.shape,y.shape)
# train model
train.fit(X1, y, epochs=1)

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)
Epoch 1/1
100000/100000 [==============================] - 77s 768us/step - loss: 0.6828 - acc: 0.7535


In [72]:
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

total, correct = 100, 0
for _ in range(total):
  X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)

  X1=X1.reshape(X1.shape[1:])
  X2=X2.reshape(X2.shape[1:])
  y = y.reshape(y.shape[1:])
  
  target = predict_sequence(train, X1)
  if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
    correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))


(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
(3, 51)
Accuracy: 98.00%


In [75]:
# spot check some examples
for _ in range(10):
  X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
  X1=X1.reshape(X1.shape[1:])
  y = y.reshape(y.shape[1:])

  target = predict_sequence(train, X1)
  print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

(3, 51)
X=[46, 39, 40, 45, 8, 24] y=[40, 39, 46], yhat=[40, 39, 46]
(3, 51)
X=[38, 20, 18, 17, 9, 26] y=[18, 20, 38], yhat=[18, 20, 38]
(3, 51)
X=[46, 3, 31, 23, 30, 48] y=[31, 3, 46], yhat=[31, 3, 46]
(3, 51)
X=[30, 10, 1, 21, 1, 43] y=[1, 10, 30], yhat=[1, 10, 30]
(3, 51)
X=[16, 43, 13, 35, 21, 15] y=[13, 43, 16], yhat=[13, 43, 16]
(3, 51)
X=[37, 36, 27, 12, 10, 1] y=[27, 36, 37], yhat=[27, 36, 37]
(3, 51)
X=[40, 34, 43, 16, 15, 2] y=[43, 34, 40], yhat=[43, 34, 40]
(3, 51)
X=[4, 17, 3, 25, 17, 21] y=[3, 17, 4], yhat=[3, 17, 4]
(3, 51)
X=[25, 16, 28, 40, 25, 14] y=[28, 16, 25], yhat=[28, 16, 25]
(3, 51)
X=[44, 18, 31, 19, 7, 40] y=[31, 18, 44], yhat=[31, 18, 44]


In [0]:
#so se desava vo generiranje recenici, ima samo encoder :) pak moze istoto da se simulira so for 1Cell, pa drug cell

#ZA UTRE probaj so 1d konvolucija bez dekoder

In [0]:
20, 5, 120 => 